# Molecular Dynamics Calculations with Quantum Espresso

When computing nonadiabatic molecular dynamics (NAMD) within the neglect-of-back-reaction approximation (NBRA), one first precomputes a molecular dynamics trajectory.

In this tutorial, we will learn how to compute a molecular dynamics trajectory using the Quantum Espresso software package. Specifically, we use the plane-wave sub-program of the Quantum Espresso package. 

Our example system will be a single TiO2 unit cell (anatase). The dynamics will be computed at the Gamma point, with a plane wave energy cut off of 30 Ry and a charge density cut off of 300 Ry. As such parameters are not converged to any particular property of interest, we warn users that such parameters are chosen only for their extreme computational efficiency, and are most likely too crude to be considered quantitative.

We will be computing just 100 fs of dynamics. That is, 100 steps with a time-step of 1 fs

## Loading the Quantum Espresso Module

In order to use Quantum Espresso, we must first load the module. To do so using the UB CCR, please execute the following command. To do so from the Jupyter Notebook, we must use the "!" mark before the command

In [1]:
!module load espresso/6.2.1

The Intel compilers are in your path. This is adequate for compiling and
running most codes. Source
/util/academic/intel/17.0/compilers_and_libraries_2017/linux/bin/compilervars.sh
for more features. 
 Intel-MPI is in your path. This is adequate for compiling and running most
codes. Source the
/util/academic/intel/17.0/compilers_and_libraries_2017/linux/mpi/intel64/bin/mpivars.sh
file for more features. 
 The MKL Libraries are in your library path. This is adequate for compiling and
running most codes. Source $MKL/bin/mklvars.sh" for more features. 


## Making the Quantum Espresso Input File

Below, will be making the Quantum Espresso input file. Given that our platform is a Jupyter Notebook, we will make the entire input file a single string, and subsequently write this string to our input file. Below, i detail the most important paramters of the input file. For a more complete description, please see the quantum espresso [pw.x input description](https://www.quantum-espresso.org/Doc/INPUT_PW.html). The pseudopotentials used herein can be found by visitin the quantum espresso [pseudopotential table](https://www.quantum-espresso.org/pseudopotentials).

&CONTROL
* `calculation` - 'md', indicates the calculation is a molecular dynamics calculations
* `nstep` - how many timesteps to take in the simulation. Each step is of size dt
* `dt` - size of each simulation step. The step size is in Rydberg atomic units
* `pseudo_dir` - location of the pseudo-potential files

&SYSTEM
* `nat` - total number of atoms in our system (6 in this case)
* `ntype` - how many different types of atoms we have (2 in this case)
* `nspin` - 1 - spin-restricted, 2 - spin-unrestricted, 4 - spin-non-colinear
* `ecutwfc` - value for the plane wave kinetic energy cutoff
* `ecutrho` - value for the charge density cutoff
* `occupations` - how the electrons will be occupying the electron states. 'smearing' - occupations will be smeared over the electornic states
* `smearing` - the type of smearing to be applied on the electronic state occupations. 'gaussian' apply the smearing in a manner consistent with a gaussian distribution centered on the highest occupied state
* `degauss` - parameter related to how the smearing on the electronic state occupations is applied. Higher degauss means more smearing. Analogous to the gaussian uncertainty.

&ELECTRONS
* `electron_maxstep` - max number of SCF cycles to reach convergence
* `conv_thr` - SCF convergence threshold

&IONS
* `ion_dynamics` - algorithm for molecular dynamics
* `ion_temperature` - Thermostat to be used for NVT dynamics
* `tempw` - temperature of the thermostat
* `nraise` - frequency of the thermostat

ATOMIC_SPECIES

The formal here is: 'Element Name' 'atomic mass' 'name of pseudo-potential file'  

K_POINTS

Define the K-point mesh for integration of the Brillouin zone. 'Gamma' selects the Gamma point (center) Another option is to define a mesh. Ex) 2 2 2 0 0 0

CELL_PARAMETERS 

Defines the cell paramters for the system 

x1, y1, z1

x2, y2, z2

x3, y3, x3

ATOMIC_POSITIONS

Defines the atomic position for the system 

Atom label, x,y,z

In [2]:
QE_input_file = \
"""
&CONTROL
  calculation   = 'md',
  nstep         = 100,
  dt            = 20.67055,
  pseudo_dir    = '/gpfs/scratch/brendan/nbra_tutorial/run_QE/',
  outdir        = './',
  prefix        = 'x0',
  disk_io       = 'low',
/

&SYSTEM
  ibrav       = 0,
  nat         = 6,
  ntyp        = 2,
  nspin       = 1,
  ecutwfc     = 30,
  ecutrho     = 300,
  occupations = 'smearing',
  smearing    = 'gaussian',
  degauss     = 0.01,
  nosym       = true,
/

&ELECTRONS
  electron_maxstep = 300,
  conv_thr         = 1.D-4,
  mixing_beta      = 0.30,
/

&IONS
  ion_dynamics      = 'verlet',
  ion_temperature   = 'andersen',
  tempw             = 300,
  nraise            = 10,
  pot_extrapolation = 'second-order',
  wfc_extrapolation = 'second-order',
/

ATOMIC_SPECIES
Ti 47.867 Ti.pbe-spn-rrkjus_psl.1.0.0.UPF
O  15.999 O.pbe-n-rrkjus_psl.1.0.0.UPF

K_POINTS gamma

CELL_PARAMETERS  angstrom
        4.6067771912         0.0000000000         0.0000000000
        0.0000000000         4.6067771912         0.0000000000
        0.0000000000         0.0000000000         2.9917566776

ATOMIC_POSITIONS angstrom
Ti    0.000000000         0.000000000         0.000000000
Ti    2.303388596         2.303388596         1.495878339
O     3.204133272         3.204133272         0.000000000
O     1.402643919         1.402643919         0.000000000
O     0.900744736         3.706032515         1.495878339
O     3.706032515         0.900744736         1.495878339
"""

## Write the Quantum Espresso Input File

Here, we simply write the Quantum Espresso Input File

In [3]:
#print (QE_input_file)
f = open("x0.md.in","w")
f.write(QE_input_file)
f.close()

## Execute the Quantum Espresso Input File

Our output file will be "x0.md.out". Execution of the following cell may take 1-2 minutes.

In [4]:
!/util/academic/espresso/6.2.1/bin/pw.x < x0.md.in > x0.md.out

## Clean Up the Unneeded Output Files

In [5]:
!sh clean.sh

rm: cannot remove ‘x0.w*’: No such file or directory
rm: cannot remove ‘sl*’: No such file or directory
rm: cannot remove ‘x0.u*’: No such file or directory
rm: cannot remove ‘CR*’: No such file or directory
